In [63]:
import sunpy.map
from sunpy.net import Fido
from sunpy.net import attrs as a
import pandas as pd

In [64]:
event_type = "FL"
tstart = "2023/01/01"
tend = "2025/7/01"
result = Fido.search(a.Time(tstart, tend),
                     a.hek.EventType(event_type),
                     a.hek.OBS.Observatory == "GOES")

In [71]:
hek_results = result["hek"]

In [72]:
print(hek_results['hgc_coord'])

        hgc_coord         
--------------------------
        POINT(5.1493818 0)
        POINT(4.7469454 0)
        POINT(3.8963418 0)
        POINT(1.2256361 0)
         POINT(0.868933 0)
          POINT(-2.7987 0)
POINT(-4.69195000000002 0)
POINT(-5.67973999999998 0)
POINT(-7.12482999999997 0)
POINT(-7.51810999999998 0)
                       ...
       POINT(-114.50454 0)
       POINT(-123.16343 0)
       POINT(-130.10339 0)
       POINT(-132.52087 0)
       POINT(-140.88552 0)
       POINT(-142.31946 0)
       POINT(-146.14328 0)
       POINT(-146.86943 0)
       POINT(-147.18195 0)
        POINT(-148.7078 0)
       POINT(-148.94679 0)
Length = 8204 rows


In [73]:
filtered_results = hek_results["event_starttime", "event_peaktime",
                               "event_endtime", "fl_goescls", "ar_noaanum", "hgc_coord"]

In [74]:
flare_df = filtered_results.to_pandas()
flare_df

,event_starttime,event_peaktime,event_endtime,fl_goescls,ar_noaanum,hgc_coord
0,2022-12-31 23:47:00,2022-12-31 23:55:00,2023-01-01 00:03:00,C1.6,0,POINT(5.1493818 0)
1,2023-01-01 00:31:00,2023-01-01 00:35:00,2023-01-01 00:41:00,C1.6,0,POINT(4.7469454 0)
2,2023-01-01 02:04:00,2023-01-01 02:28:00,2023-01-01 02:41:00,C3.7,0,POINT(3.8963418 0)
3,2023-01-01 06:56:00,2023-01-01 07:02:00,2023-01-01 07:06:00,C1.7,0,POINT(1.2256361 0)
4,2023-01-01 07:35:00,2023-01-01 07:44:00,2023-01-01 07:50:00,C1.9,0,POINT(0.868933 0)
...,...,...,...,...,...,...
8199,2025-06-30 13:20:00,2025-06-30 13:27:00,2025-06-30 13:34:00,C1.4,14130,POINT(-146.14328 0)
8200,2025-06-30 14:39:00,2025-06-30 14:50:00,2025-06-30 14:55:00,B9.6,14130,POINT(-146.86943 0)
8201,2025-06-30 15:13:00,2025-06-30 15:20:00,2025-06-30 15:26:00,C1.0,14130,POINT(-147.18195 0)
8202,2025-06-30 17:59:00,2025-06-30 18:05:00,2025-06-30 18:16:00,C1.1,14130,POINT(-148.7078 0)


In [75]:
sample = flare_df[(flare_df['event_starttime'] < '2023-08-15') & (flare_df['event_starttime'] > '2023-07-01')]


In [ ]:
# Summarize flare events
print(f"Total number of flares: {len(flare_df)}")

# Count flares by GOES class
goes_class_counts = flare_df['fl_goescls'].str[0].value_counts()
print("\nFlares by GOES class:")
print(goes_class_counts)

# Get time span
time_span = pd.to_datetime(flare_df['event_starttime'])
print(f"\nTime span: {time_span.min()} to {time_span.max()}")

# Count flares per year
flares_by_year = flare_df['event_starttime'].dt.year.value_counts().sort_index()
print("\nFlares per year:")
print(flares_by_year)

# Show most active regions
active_regions = flare_df['ar_noaanum'].value_counts().head()
print("\nMost active regions (top 5):")
print(active_regions)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 14})
# Load GOES data
goes_combined = pd.read_csv('/home/griffingoodwin/downloads/goes_combined/combined_g18_avg1m_20230701_20230815.csv')
goes_combined['time'] = pd.to_datetime(goes_combined['time'], errors='coerce')

# Load flare catalog
flare_df = pd.read_csv('/mnt/data/SDO-AIA-additional/FlareEvents/flare_events.csv')  # <-- Replace with actual path
flare_df['event_starttime'] = pd.to_datetime(flare_df['event_starttime'], errors='coerce')
flare_df['event_peaktime'] = pd.to_datetime(flare_df['event_peaktime'], errors='coerce')
flare_df['event_endtime'] = pd.to_datetime(flare_df['event_endtime'], errors='coerce')

# Plot GOES flux
plt.figure(figsize=(14, 6))
plt.plot(goes_combined['time'], goes_combined['xrsb_flux'], label='GOES XRS-B Flux', color='darkblue')

# # Overlay flares
#
# for _, flare in flare_df.iterrows():
#     start = flare['event_starttime']
#     peak = flare['event_peaktime']
#     end = flare['event_endtime']
#
#     if peak > pd.Timestamp("2023-08-15") or peak < pd.Timestamp("2023-07-01"):
#         continue
#
#     if pd.notna(start) and pd.notna(end) and start < end:
#         plt.axvspan(start.to_pydatetime(), end.to_pydatetime(), color='red', alpha=0.2)
#
#     if pd.notna(peak):
#         plt.axvline(peak.to_pydatetime(), color='red', linestyle='--', alpha=0.5)

# Final plot settings
# Add horizontal lines for flare classifications


plt.yscale('log')
plt.xlabel('Time')
plt.ylabel('Flux [W/m²]')
plt.title('GOES XRS-B Flux - Training Data')
plt.legend()
plt.tight_layout()
plt.grid(True)
plt.minorticks_on()
plt.grid(which="minor", axis="y", linestyle="--", alpha=0.5)
plt.grid(which="major", axis="y", linestyle="-", alpha=0.5)
plt.grid(which="major", axis="x", linestyle="-", alpha=0.5)
plt.gca().set_axisbelow(True)
plt.savefig('goes_flare_plot.png')
plt.show()



In [ ]:
import sunpy.map
m = sunpy.map.Map('/mnt/data2/AIA_processed_data/94/2023-08-11T03:00:00.fits')

In [ ]:
import sunpy.map
import matplotlib.pyplot as plt
from astropy.visualization import ImageNormalize, SinhStretch
import numpy as np
import astropy.units as u
from sunpy.visualization.colormaps import color_tables as ct

# Load the map
m = sunpy.map.Map('/mnt/data2/AIA_processed_data/94/2023-07-22T20:30:00.fits')

# Create a normalization with sinh stretch
norm = ImageNormalize(vmin=0, vmax=np.percentile(m.data, 99), stretch=SinhStretch())

# Ensure wavelength is in angstroms
wavelength = m.wavelength.to(u.angstrom)

# Get the appropriate AIA colormap
cmap = ct.aia_color_table(wavelength)

# Make a figure with the desired size
plt.figure(figsize=(10, 10))  # only this one matters
m.plot(norm=norm, cmap=cmap)

# Add extras
plt.title('AIA 94 Å 2023-07-22 20:30:00')
plt.tight_layout()
plt.grid(False)
plt.savefig('aia94.png', dpi=500)
plt.show()




In [ ]:
import matplotlib.pyplot as plt
from astropy.visualization import SinhStretch
from astropy.visualization.mpl_normalize import ImageNormalize

# Create a custom normalization with sinh stretch
norm = ImageNormalize(vmin=m.min(), vmax=m.max(), stretch=SinhStretch())

# Plot and save the map with sinh normalization
fig = plt.figure(figsize=(10, 10))
ax = plt.subplot(projection=m)
im = m.plot(axes=ax, norm=norm, cmap='sdoaia94')
plt.colorbar(im)
m.draw_limb()
m.draw_grid()
plt.title(f"{m.meta['WAVELNTH']} Å {m.date.strftime('%Y-%m-%d %H:%M:%S')}")
plt.savefig('map_with_sinh_norm.png', dpi=300, bbox_inches='tight')
plt.close()

In [76]:
sample

,event_starttime,event_peaktime,event_endtime,fl_goescls,ar_noaanum,hgc_coord
1558,2023-07-01 07:00:00,2023-07-01 07:06:00,2023-07-01 07:10:00,C2.5,0,POINT(132.44547 0)
1559,2023-07-01 07:18:00,2023-07-01 07:25:00,2023-07-01 07:29:00,C2.4,0,POINT(132.28002 0)
1560,2023-07-01 09:12:00,2023-07-01 09:19:00,2023-07-01 09:26:00,C2.1,0,POINT(131.23215 0)
1561,2023-07-01 11:34:00,2023-07-01 11:39:00,2023-07-01 11:44:00,C3.5,0,POINT(129.92691 0)
1562,2023-07-01 12:47:00,2023-07-01 12:56:00,2023-07-01 13:00:00,C4.1,0,POINT(129.25591 0)
...,...,...,...,...,...,...
1943,2023-08-13 09:44:00,2023-08-13 09:53:00,2023-08-13 10:15:00,C2.4,0,POINT(-77.92428 0)
1944,2023-08-13 10:35:00,2023-08-13 10:41:00,2023-08-13 10:47:00,C1.2,0,POINT(-78.39248 0)
1945,2023-08-14 02:34:00,2023-08-14 02:42:00,2023-08-14 02:50:00,C1.8,0,POINT(-87.19628 0)
1946,2023-08-14 17:43:00,2023-08-14 17:50:00,2023-08-14 17:57:00,C1.5,0,POINT(-95.54085 0)
